In [17]:
#import libraries needed for this proces
import requests
from datetime import datetime
import numpy
import time
import json
from pathlib import Path
import pandas as pd
from os.path import exists

In [18]:
##this block creates functions that can be used to extract the data from the steam API
##maybe we can add this block in the source file as a functions file

def get_request(url, parameters=None):
    #function to create an API request based on URL and  parameters
    print("Your request is being processed.")
    response = requests.get(url=url, params=parameters)
    if response:
        return response.json()
        
    else:
        print("Request unsuccessful, retrying in 5 seconds.")
        time.sleep(5)
        return get_request(url, parameters)
    time.sleep(1)



def getAppList():
    #using the function to get all data id's which include appid and app name 
    url = "https://api.steampowered.com/ISteamApps/GetAppList/v2/"
    parameters = {"request": all}
    return get_request(url, parameters=parameters)
    print("Your request has been processed successfully, call upon 'data_ids' to preview the full list of IDs and names of all currently available products on Steam as of ", datetime.now().strftime("%d/%m/%Y %H:%M:%S"), ".")
    

def getAppDetails(id):
    #function to collect the raw data from the api per app_id
    url = "http://store.steampowered.com/api/appdetails/"
    parameters = {"appids": {id}}
    raw_data = get_request(url, parameters=parameters)
    response = raw_data[str(id)]
    if(response['success']):
        data_json = response["data"]
        data_json['collection_details'] = {'created_by': 'our_scraper',
                                            'created_at': datetime.now().strftime("%d/%m/%Y %H:%M:%S")}
        f = open('..\\..\\data\\raw_data.json','a',encoding='utf-8')
        f.write(json.dumps(data_json)+'\n')
        f.close()
        return data_json
    return 
 

In [19]:
#collect all app id's from steam 
data_ids = getAppList()

#Code to collect all app id's in one list and write to a json file called app_ids 
all_app_ids = []
print("Creating a list of all IDs currently available on Steam")

for item in data_ids['applist']['apps']:
    all_app_ids.append(item['appid'])

f = open('..\\..\\data\\app_ids.json','w',encoding='utf-8')
f.write(json.dumps(all_app_ids)+'\n')
f.close()

print("List created sucessfully, the total number of IDs available on Steam is:", len(all_app_ids),".")

Your request is being processed.
Creating a list of all IDs currently available on Steam
List created sucessfully, the total number of IDs available on Steam is: 138583 .


In [ ]:
#This code can be run to collect the appids, in case there was data collected earlier in the raw_data.json file than it will add to existing files. 
counter = 0
limit = 10
keys = ['categories','package_groups','genres','screenshots','movies', 'achievements.highlighted']
Allkeys = ['ids', 'raw', 'categories','package_groups','genres','screenshots','movies', 'achievements.highlighted']
dataframes = {}


##creates dataframes for the seperate parsing keys 
for k in Allkeys:
    file = f"dataframe_{k}.xlsx"
    try:
        dataframes[k] = pd.read_excel(file)
    except:
        print(f"File doesn't exist {file}")
        dataframes[k] = pd.DataFrame()
        pass


## collect app_ids from the  app_ids file created before 
r = open('..\\..\\data\\app_ids.json','r',encoding='utf-8')
app_ids = json.load(r)

## creates an empty raw_data.json that can be appended on
f = open('..\\..\\data\\raw_data.json','a',encoding='utf-8')

# check if the json from earlier was created and set limit 
raw_exists = exists("..\\..\\data\\raw_data.json")

#create list of already collected id's 
if (raw_exists):
    #create list of collected id's 
    col_ids = []
    with open('..\\..\\data\\raw_data.json') as f: 
        for jsonObj in f: 
            rawDict = json.loads(jsonObj)
            col_ids.append(rawDict["steam_appid"])

## getting the new data
for i in app_ids: 
    if i not in col_ids:
        if(counter > limit):
            break
        id = str(i)
        counter = counter + 1
        app_data = getAppDetails(i)
        print("Processing data pertaining to Steam ID:"+id)
        if(app_data):
            print(f"Parsing id {i} with success")
            steam_appid = str(i)
            result = pd.json_normalize(app_data)
            dataframes['raw'] = pd.concat([dataframes['raw'], result])
            dataframes['ids'] = pd.concat([dataframes['ids'], pd.json_normalize({'id' : i})])

            for key in keys:
                if( key not in dataframes):
                    dataframes[key] = pd.DataFrame()
                try:
                    print(f"Parsing id {i} for key {key}")
                    data = pd.json_normalize(app_data,record_path = [key], meta = ["steam_appid"], errors="ignore")
                    dataframes[key] = pd.concat([dataframes[key],data])            
                    print(key)
                except:
                    pass                  
    else:
        print(f"Skipping id {str(i)}")
        
#writing the new data to the .xlsx files from the new dataframes
for key in Allkeys:
    print(f"Writing to dataframe {key}")
    file = f"..\\..\\data\\dataframe_{key}.xlsx"
    print(dataframes[key])
    dataframes[key].to_excel(file)


In [16]:
# create 

[612440,
 612470,
 612491,
 612500,
 612510,
 612520,
 612540,
 612570,
 612580,
 612590,
 612600,
 612610,
 612620,
 612640,
 612650,
 612660,
 612670,
 612720,
 612730,
 612740,
 612750,
 612790,
 612810,
 612820,
 612830,
 612840,
 612850,
 612870,
 612880,
 612890,
 612900,
 611960,
 611970,
 611980,
 612000,
 612010,
 612020,
 612030,
 612040,
 612050,
 612060,
 612070,
 612100,
 612110,
 612120,
 612130,
 612140,
 612150,
 612160,
 612170,
 612180,
 612190,
 612200,
 612210,
 612220,
 612230,
 612240,
 612250,
 612260,
 612261,
 612280,
 612300,
 612310,
 612330,
 612370,
 612380,
 612390,
 612400,
 612420,
 611620,
 611630,
 611640,
 611650,
 611660,
 611670,
 611690,
 611710,
 611720,
 611730,
 611740,
 611750,
 611760,
 611770,
 611790,
 611800,
 611810,
 611820,
 611830,
 611850,
 611860,
 611890,
 611950,
 611490,
 611491,
 611492,
 611493,
 611494,
 611495,
 611496,
 611497,
 611498,
 611500,
 611510,
 611511,
 611512,
 611513,
 611514,
 611520,
 611521,
 611522,
 611523,
 